In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import inspect

In [28]:
class AutoInitMeta(type):
    
    def __new__(cls, name, bases, methods):
        
        def auto_repr(instance):
            return str(instance.__dict__)
        
        cls.__repr__ = auto_repr
        return type.__new__(cls, name, bases, methods)

In [31]:
class Foo(metaclass=AutoInitMeta):
    c = 'c'
    def __init__(self, a, b=6, **kwargs):
        pass
    
    def walk(self):
        return 'in walk'

In [32]:
f = Foo(3)
f

<{'__module__': '__main__', 'c': 'c', '__init__': <function Foo.__init__ at 0x10bcca488>, 'walk': <function Foo.walk at 0x10bcca510>, '__dict__': <attribute '__dict__' of 'Foo' objects>, '__weakref__': <attribute '__weakref__' of 'Foo' objects>, '__doc__': None} at 0x10bccceb8>

In [33]:
f

<{'__module__': '__main__', 'c': 'c', '__init__': <function Foo.__init__ at 0x10bcca488>, 'walk': <function Foo.walk at 0x10bcca510>, '__dict__': <attribute '__dict__' of 'Foo' objects>, '__weakref__': <attribute '__weakref__' of 'Foo' objects>, '__doc__': None} at 0x10bccceb8>

## Select and drop

In [72]:
def select(obj, keep=None, drop=None):
    if isinstance(obj, (list, tuple)):
        idx = keep or set(range(len(obj))) - set(drop)
        return type(obj)(obj[i] for i in idx)
    elif isinstance(obj, dict):
        idx = keep or obj.keys() - set(drop)
        return {k: obj[k] for k in idx}

In [73]:
chars = list('abcdefghijklmnopqrstuvwxyz')
select(chars, [4, 1, 7])

['e', 'b', 'h']

In [74]:
select(chars, drop=[2, 5, 23, 25])

['a',
 'b',
 'd',
 'e',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y']

In [76]:
char2i = {v: k for k, v in enumerate(chars)}
select(char2i, ['e', 'd', 'z'])

{'e': 4, 'd': 3, 'z': 25}

In [79]:
out = select(char2i, drop=['b', 'c', 'y'])
for char in chars:
    if char in out:
        print(char, out[char])

a 0
d 3
e 4
f 5
g 6
h 7
i 8
j 9
k 10
l 11
m 12
n 13
o 14
p 15
q 16
r 17
s 18
t 19
u 20
v 21
w 22
x 23
z 25


## Auto repr decorator 

(limitation: decorator evaluated before object is initialized so \**kwargs can't be included unless we include everything in \_\_dict__, which may include attrs not specified in \_\_init__ call)

In [7]:
def auto_repr(cls):
    """Class decorator that provides __repr__ method automatically
    based on __init__ parameters. Unlike the repr included in AutoInit,
    this ignores kwargs.
    """
    def repr_(self):
        args = dict(inspect.signature(self.__init__).parameters)
        args.pop('kwargs', None)
        arg_strs = (f'{k}={repr(getattr(self, k))}' for k in args.keys())
        return f'{type(self).__name__}({", ".join(arg_strs)})'
    cls.__repr__ = repr_
    return cls

In [38]:
def auto_repr(cls):
    """Class decorator that provides __repr__ method automatically
    based on __init__ parameters. This aims to provide a simpler alternative 
    to AutoInit that does not require access to the arguments passed to
    __init__. Attributes will only be included in the repr if they are in 
    the class dict and appear in __init__ as a named parameter (with the
    same name).
    
    Examples
    --------
    @auto_repr
    class Foo:
        def __init__(self, a, b=6, c=None, p=0.5, **kwargs):
            self.a = a
            self.b = b
            # Different name to demonstrate that cat is not included in repr.
            self.cat = c
            # Property is not stored in class dict, not included in repr.
            self.p = p
            
        @property
        def p(self):
            return self._p
    
        @p.setter
        def p(self, val):
            if val > 0: 
                self._p = val
            else:
                raise ValueError('p must be non-negative')
            
    >>> f = Foo(3, b='b', c='c')
    >>> f
    
    Foo(a=3, b='b')
    """
    def repr_(instance):
        args = dict(inspect.signature(instance.__init__).parameters)
        arg_strs = (f'{k}={repr(v)}'for k, v in instance.__dict__.items()
                    if k in args.keys())
        return f'{type(instance).__name__}({", ".join(arg_strs)})'
    cls.__repr__ = repr_
    return cls

In [28]:
@auto_repr
class Foo:
    def __init__(self, a, b=6, d=None, p=True, **kwargs):
        self.a = a
        self.b = b
        self.d = d
        self.p = p
        
    @property
    def p(self):
        print('GETTING p')
        return self._p
    
    @p.setter
    def p(self, val):
        print('setting p')
        self._p = val
        
    def walk(self):
        return 'foo walking'

In [29]:
f = Foo(3, c='z')
f

setting p


Foo(a=3, b=6, d=None)

In [30]:
f.__dict__

{'a': 3, 'b': 6, 'd': None, '_p': True}